# 01 Webscraping Request

#### Hent data og dump

In [30]:
import bs4
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import random
import pickle
import time

workingProxyList = [] # used for proxies (free ones are too slow)
totalpageprShop  = [] # used in multiThreadGetTotalPages
allPagesRequest  = [] # used in multiThreadDownloader
reviewErrorCount = [] # used in multithreadwritecsvfile
dataDir = './data/'

In [31]:
# Denne celle skal køres, hvis der skal bruges proxies

#with open('ipList2', 'rb') as file:
#    workingProxyList = pickle.load(file)

### Functions

In [32]:
# Splitter hver url i listen, og returnere hjemmesidenavnet.
def createFiles(url):
    filename = str(url.url)
    filename = filename.split('.')

    if len(filename) == 4:
        filename = filename[2].split('/')
        filename = filename[2]
    else:
        filename = filename[3]
    return filename

In [33]:
# Henter antal review sider for hver butik i listen og gemmer det som en liste af tuples.
def multiThreadGetTotalPages(shop):
    shop = 'https://dk.trustpilot.com/review/'+shop
    progress.update(1)
    listofpages=""
    totalpages=None
    req = requests.get(shop, timeout=10)
    soup = bs4.BeautifulSoup(req.content, 'html.parser')

    try:
        totalpages = soup.find('a', attrs={'name':'pagination-button-last'}).text
        shopAndPage = {'shop': shop,'pages': totalpages}
        totalpageprShop.append(shopAndPage)
    except: 
        listofpages = [7,6,5,4,3,2,1]
        for content in listofpages:
            try:
                totalpages = soup.find('a', attrs={'name':'pagination-button-'+str(content)}).text
                if totalpages is not None:
                    shopAndPage = {'shop': shop,'pages': totalpages}
                    totalpageprShop.append(shopAndPage)
                    break
                
            except Exception as e:
                pass
           
    if len(totalpageprShop) == 0:
        shopAndPage = {'shop': shop,'pages': 1}
        totalpageprShop.append(shopAndPage)
    
    print(shopAndPage)

In [34]:
# Henter alle siderne fra trustpilot og tilføjet dem i en liste. Før noget filtering.
def multiThreadDownloader(shop,page):
    progress.update(1)
    page = page+1
    url = shop+'?page='+str(page)
    
    #Overvej at droppe proxy.. Servers er langsomme, timer ofte ud eller virker slet ikke. 
    #Selvom de er testet 5min forinden.
    if len(workingProxyList) > 0:
        for ip in workingProxyList:
            try:
                randomIpNumber = random.randint(0,len(workingProxyList))
                allPagesRequest.append(requests.get(url, timeout=15, proxies=workingProxyList[randomIpNumber]))
                break
            except:
                pass
    else:
        allPagesRequest.append(requests.get(url, timeout=10))
    
    if allpages > 5:
        time.sleep(10)

In [1]:
# Gemmer listen som indeholder alle requests data i rå format. Før filtering. 
def rawDataDump(allPagesRequest):
    dumpList = []
    tmp=""
    with tqdm(total=len(allPagesRequest)) as progress:
        for x in (allPagesRequest):
            progress.update(1)
            dumpname = createFiles(x)
            if dumpname in x.url:
                if tmp == "":
                    dumpList.append(x)
                    tmp=dumpname
                    with open(dataDir+dumpname+'.dump', 'wb') as output_file:
                        pickle.dump(dumpList,output_file)
                elif tmp==dumpname:
                    dumpList.append(x)
                    with open(dataDir+dumpname+'.dump', 'wb') as output_file:
                        pickle.dump(dumpList,output_file)   
                else:
                    dumpList = []
                    tmp=""
                    dumpList.append(x)
                    with open(dataDir+dumpname+'.dump', 'wb') as output_file:
                        pickle.dump(dumpList,output_file)

### Execute

In [36]:
# Websites du vil scrape fra på trustpilot.dk
allShopsRequest = ['www.jyskmobelfabrik.dk', 'www.cphbusiness.dk']

# Henter antal review sider for hver butik i listen og gemmer det som en liste af tuples.
with tqdm(total=len(allShopsRequest)) as progress:
    with ThreadPoolExecutor(2) as ex:
        ex.map(multiThreadGetTotalPages, allShopsRequest)

{'shop': 'https://dk.trustpilot.com/review/www.jyskmobelfabrik.dk', 'pages': '7'}
{'shop': 'https://dk.trustpilot.com/review/www.cphbusiness.dk', 'pages': '2'}



In [40]:
# Henter alle siderne fra trustpilot og tilføjer dem i en liste.
allpages=0    
for x in totalpageprShop:
    allpages = allpages+int(x['pages'])

with tqdm(total=allpages) as progress:
    for i in totalpageprShop:
        digitlist = []
        res = int(i['pages'])
        for digit in range(res):
            digitlist.append(digit)
        with ThreadPoolExecutor(1) as ex:
            ex.map(multiThreadDownloader,[i['shop']] * len(digitlist), digitlist)

In [39]:
# Gemmer den rå data indeholdende alle requests.
rawDataDump(allPagesRequest)